In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import matplotlib as mpl


# Fonts for plots
plt.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'

pd.set_option('display.max_columns', None)

# Path to data retrieval and storage
path = "C:/Users/kleinow/ownCloud/MA_Neuro"

In [2]:
# load reference data
cn_refs = pd.read_csv(path + '/cn_refs.csv')
cn_refs.head()
cn_refs.shape # (81694, 16)

C:\Users\kleinow\AppData\Local\Temp\ipykernel_6068\1242366306.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  cn_refs = pd.read_csv(path + '/cn_refs.csv')


(81694, 16)

Removal of any full duplicate rows

In [3]:
cn_refs = cn_refs.drop_duplicates().reset_index(drop=True) # considers the entire row duplicates
cn_refs.shape # (81694, 16); no full duplicate rows in the retrieved data set

(81694, 16)

Adding a column that contains the titles in lower-case

In [ ]:
cnrefs['title_lower'] = cn_refs['ref_item_title'].str.lower()   